In [ ]:
import os
os.chdir(os.environ['PROJECT_ROOT'])

In [ ]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import shap
import copy
import pdpipe as pdp
from lightgbm import LGBMRegressor
from pathlib import Path
from pandas.core.common import SettingWithCopyWarning
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from sklearn.model_selection import cross_validate, TimeSeriesSplit
from mentorship.features.kaggle.storesales.etl import ETLTransformer
from mentorship.features.history import cut_history
from mentorship.features.tuning import boruta_features_tuning
from mentorship.ml.cv.split import DateTimeSeriesSplit
from mentorship.ml.cv.util import format_cv_test_scores
from mentorship.ml.models.kaggle.storesales.hyperparams.tuning import tune_hyperparams
from mentorship.ml.models.kaggle.storesales.boosting import LGBMPipeline
from mentorship.ml.models.kaggle.storesales.linear import LinearPipeline
from mentorship.ml.models.reg import PositiveRegressor
from mentorship.ml.models.common import RecursiveTSEstimator
from mentorship.ml.models.kaggle.storesales.estimators import RecursiveTSEstimatorWithZeroCategories
from mentorship.ml.tools.kaggle.storesales.submission import make_submission_file
from mentorship.ml.tools.kaggle.storesales.curves import plot_learning_curver
from mentorship.ml.tools.timeit import timeit



%matplotlib inline
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
CV_METRICS = [
    'neg_mean_squared_log_error',
    'neg_root_mean_squared_error',
    'neg_mean_absolute_error',
    'r2'
]

In [ ]:
DATA_ROOT = Path('data', 'kaggle', 'store-sales-time-series-forecasting')

In [ ]:
train = pd.read_csv(DATA_ROOT / 'train.csv')
train.head()

In [ ]:
N_STORES = train['store_nbr'].nunique()
N_FAMILIES = train['family'].nunique()
N_TIME_SERIES = N_STORES * N_FAMILIES

DAYS_IN_YEAR = 365
N_HORIZONS = 16


# Best model

#### Features: 'store_nbr' (categorical), 'dcoilwtico', 'onpromotion', 'lag'-features (1, 2, 4, 6, 7, 14)
#### Models: LinearRegression, LGBMRegressor (depends on the family)
#### Loss function: MSLE
#### 'zero'-categories: 'baby care', 'books' -- predict constant zero

In [ ]:
# preparing the train set

lags = [1, 2, 4, 6, 7, 14]

X = train.copy()
y = X['sales'].copy()

train_transformer = ETLTransformer(lags=lags, target_col='sales')
X = train_transformer.transform(X)
splitter = DateTimeSeriesSplit()
X_train, y_train = cut_history(X=X, date_column='date', keep_interval=pd.Timedelta(days=DAYS_IN_YEAR), y=y)

X.head()

In [ ]:
plt.figure(figsize=(20, 150))
X_school = X[X['family'] == 'school and office supplies']
X_school['sales'] = np.expm1(X_school['sales'])
for i, store_nbr in enumerate(X['store_nbr'].unique()):
    plt.subplot(27, 2, i + 1)
    average_sales = X_school[X_school['store_nbr'] == store_nbr].groupby('date')['sales'].median()
    trend = average_sales.rolling(window=365, center=True, min_periods=183).mean()
    plt.ylabel('mean sales')
    ax = average_sales.plot(alpha=0.5)
    ax = trend.plot(ax=ax, linewidth=3)
    plt.title(store_nbr, color='red')

In [ ]:
# preparing the test set

test_data = pd.read_csv(DATA_ROOT / 'test.csv')
test_transformer = ETLTransformer()
test_data = test_transformer.transform(test_data)
test_data.head()

In [ ]:
### seasonality

seasonal_families = ['lingerie', 'school and office supplies']
y_seasonalities = {}
seasonal_models = {}
X['sales'] = np.expm1(X['sales'])
for current_family in X['family'].unique():
    X_current_family = X[X['family'] == current_family]
    if current_family in seasonal_families:
        average_sales_current_family = X_current_family.groupby('date')['sales'].median()
        X_current_family_seasonal = X_current_family.drop_duplicates(subset=['date'])
        y_current_family = y.loc[X_current_family_seasonal.index]

        X_current_family_dates = X_current_family_seasonal['date']
        test_current_family_dates = test_data[test_data['family'] == current_family].drop_duplicates(subset=['date'])['date']
        train_and_test_dates = pd.concat([X_current_family_dates, test_current_family_dates])
        train_and_test_dates = pd.to_datetime(train_and_test_dates).dt.to_period('D')
        
        fourier = CalendarFourier(freq='A', order=10)  # 10 sin/cos pairs for "A"nnual seasonality
        dp_train_and_test = DeterministicProcess(index=train_and_test_dates, constant=True, additional_terms=[fourier], 
                                                 drop=True)
        
        X_current_family_train_season = dp_train_and_test.in_sample()[:-N_HORIZONS]
        X_current_family_test_season = dp_train_and_test.in_sample()[-N_HORIZONS:]
        seasonal_models[current_family] = (LinearRegression().fit(X_current_family_train_season, average_sales_current_family), 
                                           X_current_family_test_season)
        y_fit_current_family = pd.DataFrame(data={'season': seasonal_models[current_family][0].predict(X_current_family_train_season), 
                                                  'date': X_current_family['date'].unique()})
        X_current_family = X_current_family.reset_index().merge(y_fit_current_family, how='left', on='date').set_index('index')
        y_seasonalities[current_family] = X_current_family['season']
    else:
        y_seasonalities[current_family] = pd.Series(data=0, index=X_current_family.index)
y_season = pd.concat(y_seasonalities.values()).loc[X.index]

In [ ]:
linear_categories = ['baby care', 'books', 'lawn and garden', 'home appliances']
fit_params = {'feature_name': ['store_nbr', 'onpromotion', 'dcoilwtico', 'lag_1', 'lag_2', 'lag_4', 'lag_6', 'lag_7', 'lag_14'],
              'categorical_feature': ['store_nbr']}

In [ ]:
### trend

# linear_trend_families = ['beverages', 'eggs', 'grocery i', 'meats', 'pet supplies', 'produce']
# quadr_trend_families = ['grocery ii', 'prepared foods']
# y_trends = {}
# trend_models = {}
# X['sales'] = np.expm1(X['sales'])
# for current_family in X['family'].unique():
#     X_current_family = X[X['family'] == current_family]
#     if current_family in linear_trend_families + quadr_trend_families:
#         average_sales_current_family = X_current_family.groupby('date')['sales'].median()
#         X_current_family_trend = X_current_family.drop_duplicates(subset=['date'])
#         y_current_family = y.loc[X_current_family_trend.index]

#         if current_family in linear_trend_families:
#             dp_train_and_test = DeterministicProcess(index=list(y_current_family.index) + list(range(N_HORIZONS)), order=1, drop=True)
#         else:
#             dp_train_and_test = DeterministicProcess(index=list(y_current_family.index) + list(range(N_HORIZONS)), order=2, drop=True)

#         X_current_family_train_trend = dp_train_and_test.in_sample()[:-N_HORIZONS]
#         X_current_family_test_trend = dp_train_and_test.in_sample()[-N_HORIZONS:]
#         trend_models[current_family] = (LinearRegression().fit(X_current_family_train_trend, average_sales_current_family), 
#                                         X_current_family_test_trend)
#         y_fit_current_family = pd.DataFrame(data={'trend': trend_models[current_family][0].predict(X_current_family_train_trend), 
#                                                   'date': X_current_family['date'].unique()})
#         X_current_family = X_current_family.reset_index().merge(y_fit_current_family, how='left', on='date').set_index('index')
#         y_trends[current_family] = X_current_family['trend']
#     else:
#         y_trends[current_family] = pd.Series(data=0, index=X_current_family.index)
# y_trend = pd.concat(y_trends.values()).loc[X.index]

In [ ]:
plt.figure(figsize=(8, 3))
X_school = X[X['family'] == 'school and office supplies']
average_sales = X_school.groupby('date')['sales'].median()
plt.ylabel('mean sales')
ax = average_sales.plot(alpha=0.5)
ax = X_school.groupby('date')['season'].median().plot(ax=ax, linewidth=3)
plt.title('school and office supplies', color='red')

In [ ]:
X.loc[:, 'sales_deseasonalized'] = X['sales'] - y_season
y_deseasonalized = X['sales'] - y_season

In [ ]:
plt.figure(figsize=(20, 130))
for i, family in enumerate(X['family'].unique()):
    plt.subplot(17, 2, i + 1)
    average_sales = X[X['family'] == family].groupby('date')['sales_deseasonalized'].median()
    trend = average_sales.rolling(window=365, center=True, min_periods=183).mean()
    plt.ylabel('mean sales')
    ax = average_sales.plot(alpha=0.5)
    ax = trend.plot(ax=ax, linewidth=3)
    plt.title(family, color='red')

In [ ]:
not_deseasonalized_families_ind = X[~X['family'].isin(seasonal_families)].index
X.loc[not_deseasonalized_families_ind, 'sales_deseasonalized'] = np.log1p(X.loc[not_deseasonalized_families_ind, 'sales_deseasonalized'])

In [ ]:
for current_lag in lags:
    X.loc[:, 'lag_{}'.format(current_lag)] = X.groupby(['store_nbr', 'family'])['sales_deseasonalized'].shift(current_lag)

In [ ]:
# importing best cross-validation and final params
import json

with open(DATA_ROOT / 'best_cv_params.txt') as f:
    data = f.read()
best_cv_params = json.loads(data)

with open(DATA_ROOT / 'best_final_params.txt') as f:
    data = f.read()
best_final_params = json.loads(data)
        
with open(DATA_ROOT / 'best_cv_params_detrended.txt') as f:
    data = f.read()
best_cv_params_detrended = json.loads(data)

with open(DATA_ROOT / 'best_final_params_detrended.txt') as f:
    data = f.read()
best_final_params_detrended = json.loads(data)

with open(DATA_ROOT / 'best_cv_params_deseasonalized.txt') as f:
    data = f.read()
best_cv_params_deseasonalized = json.loads(data)

with open(DATA_ROOT / 'best_final_params_deseasonalized.txt') as f:
    data = f.read()
best_final_params_deseasonalized = json.loads(data)

In [ ]:
tscv_inner = TimeSeriesSplit(gap=0, max_train_size=(DAYS_IN_YEAR - 4 * N_HORIZONS) * N_STORES, n_splits=4,
                             test_size=N_HORIZONS * N_STORES)
train_indices = next(splitter.split(X, y))[0]

X_train_first_fold, y_train_first_fold = X.iloc[train_indices], y_deseasonalized.iloc[train_indices]

X_train_deseasonalized_families = X_train_first_fold[X_train_first_fold['family'].isin(seasonal_families)]
y_train_deseasonalized_families = y_train_first_fold.loc[X_train_deseasonalized_families.index]

best_cv_params_deseasonalized = tune_hyperparams(X_train_deseasonalized_families.drop(columns=['sales']), 
                                                 y_train_deseasonalized_families, tscv_inner=tscv_inner, lags=lags, 
                                                 level='store_nbr', target_col='sales_deseasonalized', 
                                                 zero_categories=['baby care', 'books'], predict_negative=True, 
                                                 use_final_metric=False, scoring='neg_root_mean_squared_error')

In [ ]:
base_pipelines = {}
for current_family in X['family'].unique():
    if current_family in linear_categories:
        base_pipelines[current_family] = LinearPipeline(cols_to_scale=['dcoilwtico'], cols_to_encode=['store_nbr'], 
                                                        drop_columns=['onpromotion'], lags=lags, split_key='family', 
                                                        target_col='sales_deseasonalized', level='store_nbr')
    else:
        if current_family in seasonal_families:
            base_pipelines[current_family] = LGBMPipeline(lags=lags, split_key='family', target_col='sales_deseasonalized', 
                                                          level='store_nbr', params=best_cv_params_deseasonalized[current_family], 
                                                          fit_params=fit_params, use_final_metric=False, predict_negative=True)
        else:
            base_pipelines[current_family] = LGBMPipeline(lags=lags, split_key='family', target_col='sales_deseasonalized', 
                                                          level='store_nbr', params=best_cv_params[current_family],
                                                          fit_params=fit_params, use_final_metric=True, predict_negative=False)

modelling_pipeline = RecursiveTSEstimatorWithZeroCategories(split_key='family', target_col='sales_deseasonalized', 
                                                            trend_pred=y_season, base_pipelines=base_pipelines,
                                                            zero_categories=['baby care', 'books'], y_detrended=y_deseasonalized)

scores = cross_validate(modelling_pipeline, X.drop(columns=['sales']), y, cv=splitter,
                        scoring=CV_METRICS, return_estimator=True, error_score='raise', n_jobs=-1)
format_cv_test_scores(scores, metrics_to_plot=['root_mean_squared_log_error'])

In [ ]:
scores_for_every_family = {}
tscv = TimeSeriesSplit(gap=0, max_train_size=DAYS_IN_YEAR * N_STORES, n_splits=4, test_size=N_HORIZONS * N_STORES)

for current_family in X['family'].unique():
    X_current_family = X[X['family'] == current_family]
    y_current_family = y.loc[X_current_family.index]
    
    base_pipeline = {current_family:base_pipelines[current_family]}
    modelling_pipeline = RecursiveTSEstimatorWithZeroCategories(split_key='family', target_col='sales_deseasonalized', 
                                                                trend_pred=y_season, base_pipelines=base_pipeline,
                                                                zero_categories=['baby care', 'books'], y_detrended=y_deseasonalized)
    
    scores = cross_validate(modelling_pipeline, X_current_family.drop(columns=['sales']), y_current_family, cv=tscv, 
                            scoring=CV_METRICS, return_estimator=True, error_score='raise', n_jobs=-1)

    scores_for_every_family[current_family] = format_cv_test_scores(scores, save_scores=True, print_scores=False)

In [ ]:
# best model after deseasonalizing

plot_scores_for_every_feature_value(scores_for_every_family, 'family')

In [ ]:
X_train, y_train = cut_history(X=X, date_column='date', keep_interval=pd.Timedelta(days=DAYS_IN_YEAR), y=y_deseasonalized)
tscv_inner = TimeSeriesSplit(gap=0, max_train_size=(DAYS_IN_YEAR - 4 * N_HORIZONS) * N_STORES, n_splits=4,
                             test_size=N_HORIZONS * N_STORES)

X_train_deseasonalized_families = X_train[X_train['family'].isin(seasonal_families)]
y_train_deseasonalized_families = y_train.loc[X_train_deseasonalized_families.index]

best_final_params_deseasonalized = tune_hyperparams(X_train_deseasonalized_families.drop(columns=['sales']), 
                                                    y_train_deseasonalized_families, tscv_inner=tscv_inner, lags=lags, 
                                                    level='store_nbr', target_col='sales_deseasonalized', 
                                                    zero_categories=['baby care', 'books'], predict_negative=True, 
                                                    use_final_metric=False, scoring='neg_root_mean_squared_error')

In [ ]:
# training and testing the final model; saving the predictions

X_train, y_train = cut_history(X=X, date_column='date', keep_interval=pd.Timedelta(days=DAYS_IN_YEAR), y=y)

base_pipelines = {}
for current_family in X['family'].unique():
    if current_family in linear_categories:
        base_pipelines[current_family] = LinearPipeline(cols_to_scale=['dcoilwtico'], cols_to_encode=['store_nbr'], 
                                                        drop_columns=['onpromotion', 'sales_deseasonalized'], lags=lags, 
                                                        split_key='family', target_col='sales_deseasonalized', level='store_nbr')
    else:
        if current_family in seasonal_families:
            base_pipelines[current_family] = LGBMPipeline(lags=lags, split_key='family', target_col='sales_deseasonalized', 
                                                          level='store_nbr', params=best_final_params_deseasonalized[current_family], 
                                                          fit_params=fit_params, use_final_metric=False, predict_negative=True)
        else:
            base_pipelines[current_family] = LGBMPipeline(lags=lags, split_key='family', target_col='sales_deseasonalized', 
                                                          level='store_nbr', params=best_final_params[current_family], 
                                                          fit_params=fit_params, use_final_metric=True, predict_negative=False)

final_modelling_pipeline = RecursiveTSEstimatorWithZeroCategories(split_key='family', target_col='sales_deseasonalized',
                                                                  base_pipelines=base_pipelines, y_detrended=y_deseasonalized,
                                                                  zero_categories=['baby care', 'books'], 
                                                                  detrended_categories=seasonal_families, 
                                                                  trend_models=seasonal_models)

final_modelling_pipeline.fit(X_train.drop(columns=['sales']), y_train)
make_submission_file(test_data, final_modelling_pipeline, 'optuna_LGBM_and_linreg.csv')

### Score on Kaggle: 0.4185

#### Compare scores for every family

In [ ]:
base_pipelines = {}
linear_categories = ['baby care', 'books', 'lawn and garden', 'home appliances']
params = {'categorical_feature': 'name: "store_nbr"'}
zero_categories = ['baby care', 'books']

for current_family in X['family'].unique():
    if current_family in linear_categories:
        base_pipelines[current_family] = LinearPipeline(cols_to_scale=['dcoilwtico'], cols_to_encode=['store_nbr'], 
                                                        drop_columns=['onpromotion'], lags=lags, split_key='family', 
                                                        target_col='sales', level=['store_nbr'])
    else:
        base_pipelines[current_family] = LGBMPipeline(lags=lags, split_key='family', target_col='sales', level=['store_nbr'],
                                                      params=best_cv_params[current_family].update(params))

In [ ]:
scores_for_every_family = {}
tscv = TimeSeriesSplit(gap=0, max_train_size=DAYS_IN_YEAR * N_STORES, n_splits=4, test_size=N_HORIZONS * N_STORES)

for current_family in X['family'].unique():
    X_current_family = X[X['family'] == current_family]
    y_current_family = y.loc[X_current_family.index]
    
    base_pipeline = {current_family:base_pipelines[current_family]}
    modelling_pipeline = RecursiveTSEstimatorWithZeroCategories(base_pipelines=base_pipeline, split_key='family', 
                                                                target_col='sales', zero_categories=zero_categories)
    
    scores = cross_validate(modelling_pipeline, X_current_family, y_current_family, cv=tscv, scoring=CV_METRICS, 
                            return_estimator=True, error_score='raise', n_jobs=-1)

    scores_for_every_family[current_family] = format_cv_test_scores(scores, save_scores=True, print_scores=False)

In [ ]:
def plot_scores_for_every_feature_value(scores, feature):
    rmsle_for_every_feature_value = {}
    for key in scores.keys():
        rmsle_for_every_feature_value[key] = round(scores[key][0]["root_mean_squared_log_error"], 3)

    sorted_rmsle_for_every_feature_value = {str(k): v for k, v in sorted(rmsle_for_every_feature_value.items(), 
                                                                    key=lambda item: item[1], reverse=True)}
    plt.figure(figsize=(20, 5))
    plt.bar(sorted_rmsle_for_every_feature_value.keys(), sorted_rmsle_for_every_feature_value.values())
    plt.xticks(rotation=90)
    plt.axhline(y=0.4185, color='red', linestyle='dashed')
    plt.title(feature)
    plt.show()

In [ ]:
# best model before deseasonalizing

plot_scores_for_every_feature_value(scores_for_every_family, 'family')

<b><p>1) Model probably makes mistakes in 'grocery ii' category, because of the distribution of its sales (it rises up sharply in last months); in 'school and office supplies' category (there is a period in the last year with extremely high sales, which could affect on the boosting algorithm).</p>
    <p><i>Possible solution:</i> for 'grocery ii' category train the model only on the recent 'high values' data (last months); for 'school and office supplies' use some seasonality tools or other model (Prophet, for example) (it can be seen that sales peaks are seasonal, which is logical for such category).</p></b>

In [ ]:
last = next(splitter.split(X, y))
for last in splitter.split(X, y):
    continue
    
train_indices = last[0]

X_train_last_fold, y_train_last_fold = X.iloc[train_indices], y.iloc[train_indices]

In [ ]:
plt.figure(figsize=(30, 90))
for i, current_family in enumerate(X['family'].unique()):
    X_current_family = X_train_last_fold[X_train_last_fold['family'] == current_family].drop(columns=['date', 'sales', 'family'])
    y_current_family = y_train_last_fold.loc[X_current_family.index]
    
    plt.subplot(11, 3, i + 1)
    lgbm_model = lgb.LGBMRegressor(importance_type='gain')
    lgbm_model.fit(X_current_family, y_current_family, categorical_feature=['store_nbr'])
    lgbm_feature_imp = pd.DataFrame(sorted(zip(lgbm_model.feature_importances_, X_current_family.columns)), 
                                    columns=['Value', 'Feature'])
    sns.barplot(x='Value', y='Feature', data=lgbm_feature_imp.sort_values(by='Value', ascending=False))
    plt.title(f'{current_family} (feature_importances_)')

<b><p>2) As we can see on these plots, features 'lag_2', 'lag_4', 'lag_6' aren't very useful in most categories (by LGBM feature_importance). Probably it could be useful to try some categories without some of these 3 features.</p>
   <p>P.S. We don't pay attention to linear categories here.</b>

In [ ]:
for i, current_family in enumerate([family for family in X['family'].unique() if family not in linear_categories]):
    print(current_family)
    X_current_family = X_train_last_fold[X_train_last_fold['family'] == current_family].drop(columns=['date', 'sales', 'family'])
    y_current_family = y_train_last_fold.loc[X_current_family.index]
    
    lgbm_model = lgb.LGBMRegressor()
    lgbm_model.fit(X_current_family, np.log1p(y_current_family), categorical_feature=['store_nbr'])
    explainer = shap.TreeExplainer(lgbm_model)
    shap_values_current_family = explainer(X_current_family)
    shap.summary_plot(shap_values_current_family)

<b><p>3) Most of 'bad' categories aren't first need goods, whereas most of 'good' categories are first need goods. Because of this fact, one thing about Shap plots becomes clear: most of 'bad' categories have 'store_nbr' feature as the most important (by SHAP), whereas most of 'good' categories don't have such trend. It can be explained in this way: predictions of categories, which can't be named as first need, can be influenced by store (probably, somewhere such goods aren't sold at all), whereas first need goods are sold in most of stores. What can we do with this? Probably it could be helpful to try first need categories without 'store_nbr' feature.</p>
   <p>4) Most of 'good' categories have a lot of samples where low 'onpromotion' values have very low shapley values. It seems logical: less promos -- less sales number. But most of 'bad' categories don't have such tendency. Low values of onpromotion feature mostly have negative shapley values, but not so low as in most of 'good' categories.</p></b>

In [ ]:
def scores_for_every_feature_value(X, y, feature, base_pipelines):
    scores_for_every_feature_value = {}
    zero_categories = ['baby care', 'books']
    for current_feature_value in X[feature].unique():
        X_current_feature_value = X[X[feature] == current_feature_value]
        if feature != 'store_nbr':
            X_current_feature_value = X_current_feature_value.drop(columns=[feature])
        y_current_feature_value = y.loc[X_current_feature_value.index]
        
        modelling_pipeline = RecursiveTSEstimatorWithZeroCategories(base_pipelines=base_pipelines, split_key='family', 
                                                                    target_col='sales', zero_categories=zero_categories)
    
        scores = cross_validate(modelling_pipeline, X_current_feature_value, y_current_feature_value, cv=splitter, 
                                scoring=CV_METRICS, return_estimator=True, error_score='raise', n_jobs=-1)

        scores_for_every_feature_value[current_feature_value] = format_cv_test_scores(scores, save_scores=True, 
                                                                                      print_scores=False)
    return scores_for_every_feature_value

#### Compare scores for every store_nbr

In [ ]:
scores_for_every_store_nbr = scores_for_every_feature_value(X, y, feature='store_nbr', base_pipelines=base_pipelines)

In [ ]:
plot_scores_for_every_feature_value(scores_for_every_store_nbr, 'store_nbr')

In [ ]:
plt.figure(figsize=(30, 200))
for i, family in enumerate(X['family'].unique()):
    plt.subplot(33, 3, 3 * i + 1)
    average_sales = X[(X['family'] == family) & (X['store_nbr'] == 52)].groupby('date')['sales'].median()
    trend = average_sales.rolling(window=365, center=True, min_periods=183).mean()
    plt.ylabel('mean sales')
    ax = average_sales.plot(alpha=0.5)
    ax = trend.plot(ax=ax, linewidth=3)
    plt.title(family, color='red')
    
    plt.subplot(33, 3, 3 * i + 2)
    average_sales = X[(X['family'] == family) & (X['store_nbr'] == 18)].groupby('date')['sales'].median()
    trend = average_sales.rolling(window=365, center=True, min_periods=183).mean()
    plt.ylabel('mean sales')
    ax = average_sales.plot(alpha=0.5)
    ax = trend.plot(ax=ax, linewidth=3)
    plt.title(family, color='red')
    
    plt.subplot(33, 3, 3 * i + 3)
    average_sales = X[(X['family'] == family) & (X['store_nbr'] == 25)].groupby('date')['sales'].median()
    trend = average_sales.rolling(window=365, center=True, min_periods=183).mean()
    plt.ylabel('mean sales')
    ax = average_sales.plot(alpha=0.5)
    ax = trend.plot(ax=ax, linewidth=3)
    plt.title(family, color='red')

<b><p>5) Model makes mistakes in 52-nd store_nbr, because of the distribution of its sales (no data before ~2017 year, probably this store was opened very late); in 18-th, 25-th store_nbr because of no data during some period of time in the last year of observations (there is no such period in other categories).</p>
    <p><i>Possible solution:</i> for 52-nd store_nbr train the model only on the recent non-zero data (since the store was opened), for 18-th, 25-th store_nbr fill 'no-data' samples using some rolling.</p></b>

#### Compare scores for every store_city

In [ ]:
X_copy = X.copy()
X_copy = train_transformer.add_store_features(X_copy, columns_to_add=['city'])
X_copy['store_city'] = LabelEncoder().fit_transform(X_copy['store_city']) + 1
X_copy.head()

In [ ]:
scores_for_every_store_city = scores_for_every_feature_value(X_copy, y, feature='store_city', base_pipelines=base_pipelines)

In [ ]:
plot_scores_for_every_feature_value(scores_for_every_store_city, 'store_city')

In [ ]:
plt.figure(figsize=(30, 200))
for i, family in enumerate(X_copy['family'].unique()):
    plt.subplot(33, 3, 3 * i + 1)
    average_sales = X_copy[(X_copy['family'] == family) & (X_copy['store_city'] == 21)].groupby('date')['sales'].median()
    trend = average_sales.rolling(window=365, center=True, min_periods=183).mean()
    plt.ylabel('mean sales')
    ax = average_sales.plot(alpha=0.5)
    ax = trend.plot(ax=ax, linewidth=3)
    plt.title(f'{family}(21)', color='red')
    
    plt.subplot(33, 3, 3 * i + 2)
    average_sales = X_copy[(X_copy['family'] == family) & (X_copy['store_city'] == 15)].groupby('date')['sales'].median()
    trend = average_sales.rolling(window=365, center=True, min_periods=183).mean()
    plt.ylabel('mean sales')
    ax = average_sales.plot(alpha=0.5)
    ax = trend.plot(ax=ax, linewidth=3)
    plt.title(f'{family}(15)', color='red')
    
    plt.subplot(33, 3, 3 * i + 3)
    average_sales = X_copy[(X_copy['family'] == family) & (X_copy['store_city'] == 7)].groupby('date')['sales'].median()
    trend = average_sales.rolling(window=365, center=True, min_periods=183).mean()
    plt.ylabel('mean sales')
    ax = average_sales.plot(alpha=0.5)
    ax = trend.plot(ax=ax, linewidth=3)
    plt.title(f'{family}(7)', color='red')

<b><p>6) Big mistakes in 21-st and 15-th store_city can be explained by their distribution: 21-st store_city has some months with missing data (last year), 15-th store_city has an extremely fast sales raising in the last months of observations.</p>
   <p>Possible solution: fill missing data with some rolling for 21-st store_city; use only last months for training for 15-th store_city.</b>

#### Compare scores for every store_cluster

In [ ]:
X_copy = X.copy()
X_copy = train_transformer.adding_stores_data(X_copy, columns_to_add=['cluster'])
X_copy.head()

In [ ]:
scores_for_every_store_cluster = scores_for_every_feature_value(X_copy, y, feature='store_cluster', base_pipelines=base_pipelines)
plot_scores_for_every_feature_value(scores_for_every_store_cluster, 'store_cluster')

In [ ]:
stores_data = pd.read_csv(DATA_ROOT / 'stores.csv')
stores_data['city'] = LabelEncoder().fit_transform(stores_data['city']) + 1
stores_data['type'] = LabelEncoder().fit_transform(stores_data['type']) + 1
# stores_data[(stores_data['cluster'] == 11) | (stores_data['cluster'] == 16) | (stores_data['cluster'] == 1)]

stores_data[(stores_data['city'] == 21) | (stores_data['city'] == 15)]

<b><p>7) Big mistakes in some clusters can be explained by the fact that every such cluster has 'bad' 'store_nbr' or 'bad' 'store_city' in their composition. To avoid such mistakes it is necessary to deal with 'store_nbr' and 'store_city' first.</p></b>

## Top-N worst time series

In [ ]:
scores_for_every_ts = {}
tscv_every_ts = TimeSeriesSplit(gap=0, max_train_size=DAYS_IN_YEAR, n_splits=4, test_size=N_HORIZONS)

for current_store_nbr in X['store_nbr'].unique():
    for current_family in X['family'].unique():
        print(current_store_nbr, current_family)
        X_current_ts = X[(X['store_nbr'] == current_store_nbr) & (X['family'] == current_family)]
        y_current_ts = y.loc[X_current_ts.index]
        
        base_pipeline = {current_family:base_pipelines[current_family]}
        modelling_pipeline = RecursiveTSEstimatorWithZeroCategories(base_pipelines=base_pipeline, split_key='family', 
                                                                    target_col='sales', zero_categories=zero_categories)
    
        scores = cross_validate(modelling_pipeline, X_current_ts, y_current_ts, cv=tscv_every_ts, scoring=CV_METRICS, 
                                return_estimator=True, error_score='raise', n_jobs=-1)

        scores_for_every_ts[f'{current_store_nbr}, {current_family}'] = format_cv_test_scores(scores, save_scores=True, 
                                                                                         print_scores=False)

In [ ]:
import json

with open(DATA_ROOT / 'every_ts_scores.txt') as f:
    data = f.read()
scores_for_every_ts = json.loads(data)

In [ ]:
rmsle_scores_every_ts = {k: v[0]['root_mean_squared_log_error'] for k, v in scores_for_every_ts.items()}
rmsle_sorted_scores_every_ts = {k: v for k, v in sorted(rmsle_scores_every_ts.items(), 
                                                        key=lambda item: item[1], reverse=True)}

In [ ]:
bad_rmsle_scores_ts = {k: v for k, v in rmsle_sorted_scores_every_ts.items() if v > 0.4185}
bad_rmsle_scores_ts = {tuple(k.split(', ')): v for k, v in bad_rmsle_scores_ts.items()}
print(len(bad_rmsle_scores_ts))
bad_rmsle_scores_ts

In [ ]:
# time series with bad store_nbr (explained above)

len([k for k in bad_rmsle_scores_ts.keys() if k[0] in ['52', '18', '25']])

In [ ]:
bad_rmsle_scores_ts = {k: v for k, v in bad_rmsle_scores_ts.items() if k[0] not in ['52', '18', '25']}

In [ ]:
# time series with two bad (and explained why) categories: 'school and office supplies', 'grocery ii'

len([k for k in bad_rmsle_scores_ts.keys() if k[1] in ['school and office supplies', 'grocery ii']])

In [ ]:
bad_rmsle_scores_ts = {k: v for k, v in bad_rmsle_scores_ts.items() if k[1] not in ['school and office supplies', 'grocery ii']}

In [ ]:
# zero categories

print('books:', len([k for k in bad_rmsle_scores_ts.keys() if k[1] == 'books']))
print('baby care:', len([k for k in bad_rmsle_scores_ts.keys() if k[1] == 'baby care']))

In [ ]:
bad_rmsle_scores_ts = {k: v for k, v in bad_rmsle_scores_ts.items() if k[1] not in ['books', 'baby care']}

In [ ]:
# time series with stores from 'bad' cities (also explained)

stores_data = pd.read_csv(DATA_ROOT / 'stores.csv')
stores_data['city'] = LabelEncoder().fit_transform(stores_data['city'])
len([k for k in bad_rmsle_scores_ts.keys() if int(k[0]) in stores_data[stores_data['city'].isin([21, 15])]['store_nbr'].unique()])

In [ ]:
bad_rmsle_scores_ts = {k: v for k, v in bad_rmsle_scores_ts.items() if int(k[0]) not in stores_data[stores_data['city'].isin([21, 15])]['store_nbr'].unique()}
len(bad_rmsle_scores_ts)

In [ ]:
# Let's leave only those time series, where rmsle > 0.66 (because probably these (bigger) mistakes 
# are the most important in those 569 which are left)
# 0.66 is the bigger average mistake by category (lingerie)

bad_rmsle_scores_ts = {k: v for k, v in bad_rmsle_scores_ts.items() if v > 0.66}
len(bad_rmsle_scores_ts)

In [ ]:
bad_categories = pd.Series([k[1] for k in bad_rmsle_scores_ts.keys()])
bad_categories.value_counts()

In [ ]:
bad_rmsle_scores_ts_sorted = {k: v for k, v in sorted(bad_rmsle_scores_ts.items(), 
                                                      key=lambda item: item[0][1])}

In [ ]:
for current_ts in bad_rmsle_scores_ts_sorted.keys():
    print(f'{current_ts[0]}, {current_ts[1]}')
    X_current_ts = X_train_last_fold[(X_train_last_fold['family'] == current_ts[1]) & 
                                     (X_train_last_fold['store_nbr'] == int(current_ts[0]))].drop(columns=['date', 'sales', 'family', 'store_nbr'])
    y_current_ts = y_train_last_fold.loc[X_current_ts.index]
    
    lgbm_model = lgb.LGBMRegressor()
    lgbm_model.fit(X_current_ts, np.log1p(y_current_ts))
    explainer = shap.TreeExplainer(lgbm_model)
    shap_values_current_ts = explainer(X_current_ts)
    shap.summary_plot(shap_values_current_ts)

<b><p>8) 'onpromotion' feature has a very small impact on model output (by SHAP) of the 'celebration' category in these summary_plots. It was shown above that LGBM feature_importance of 'onpromotion' for this category was also very small. Probably, this feature is the reason, why model makes mistakes on 'celebration' category and it is necessary to remove this feature for this category.</p>
   <p><p>   The same situation with 'ladieswear', 'lawn and garden', 'lingerie' (in most stores) categories.</p>

In [ ]:
X_train_last_fold[X_train_last_fold['onpromotion'] > 0]['family'].nunique()

In [ ]:
X_train_last_fold[(X_train_last_fold['onpromotion'] > 0) & (X_train_last_fold['family'] == 'hardware')].shape

### Waterfall plots for 'lingerie' category

In [ ]:
X_lingerie = X_train_last_fold[X_train_last_fold['family'] == 'lingerie'].drop(columns=['date', 'sales', 'family'])
y_lingerie = y_train_last_fold.loc[X_lingerie.index]
lgbm_model = lgb.LGBMRegressor()
lgbm_model.fit(X_lingerie, np.log1p(y_lingerie), categorical_feature=['store_nbr'])
explainer = shap.TreeExplainer(lgbm_model)
shap_values_lingerie = explainer(X_lingerie)
shap.summary_plot(shap_values_lingerie)

In [ ]:
X_lingerie = X_train_last_fold[X_train_last_fold['family'] == 'lingerie'].reset_index()
random_samples = X_lingerie.sample(n=50)
for current_sample in random_samples.index:
    print('date:', random_samples['date'][current_sample])
    print('store_nbr:', random_samples['store_nbr'][current_sample])
    print('index:', current_sample)
    shap.plots.waterfall(shap_values_lingerie[current_sample])

### Changes in the best model according to the error analysis

In [ ]:
# filling missing values for 18-th and 25-th store_nbr using linear regression predictions

X_nan_inds = X[X.isnull().any(axis=1)].index
X = X.dropna().reset_index(drop=True)
y = y.drop(index=X_nan_inds).reset_index(drop=True)

missing_data_18 = X[(X['store_nbr'] == 18) & (X['date'] >= '2016-08-15') & (X['date'] <= '2016-12-02')].index
missing_data_25 = X[(X['store_nbr'] == 25) & (X['date'] >= '2016-08-22') & (X['date'] <= '2016-10-26')].index


base_pipelines_18_25 = {}
for current_family in X['family'].unique():
    base_pipelines_18_25[current_family] = LinearPipeline(cols_to_scale=['dcoilwtico'], drop_columns=['onpromotion'], 
                                                          lags=lags, split_key='family', target_col='sales', level='store_nbr')

linear_pipeline = RecursiveTSEstimatorWithZeroCategories(split_key='family', target_col='sales', 
                                                         base_pipelines=base_pipelines_18_25, 
                                                         zero_categories=['baby care', 'books'])
    
X_train_linreg_18, X_train_linreg_25 = X[(X['store_nbr'] == 18) & (X['date'] >= '2015-08-15') & (X['date'] < '2016-08-15')], \
                                       X[(X['store_nbr'] == 25) & (X['date'] >= '2015-08-22') & (X['date'] < '2016-08-22')]
y_train_linreg_18, y_train_linreg_25 = y.loc[X_train_linreg_18.index], y.loc[X_train_linreg_25.index]
X_test_linreg_18, X_test_linreg_25 = X.loc[missing_data_18], X.loc[missing_data_25]

linear_pipeline.fit(X_train_linreg_18, y_train_linreg_18)
preds_18 = linear_pipeline.predict(X_test_linreg_18)
X.loc[missing_data_18, 'sales'] = np.log1p(preds_18)
y.loc[missing_data_18] = preds_18

linear_pipeline.fit(X_train_linreg_25, y_train_linreg_25)
preds_25 = linear_pipeline.predict(X_test_linreg_25)
X.loc[missing_data_25, 'sales'] = np.log1p(preds_25)
y.loc[missing_data_25] = preds_25

In [ ]:
for current_lag in lags:
    X.loc[:, 'lag_{}'.format(current_lag)] = X.groupby(['store_nbr', 'family'])['sales'].shift(current_lag)

In [ ]:
tscv_inner = TimeSeriesSplit(gap=0, max_train_size=(DAYS_IN_YEAR - 4 * N_HORIZONS) * N_STORES, n_splits=4,
                             test_size=N_HORIZONS * N_STORES)
train_indices = next(splitter.split(X, y))[0]
X_train_first_fold, y_train_first_fold = X.iloc[train_indices], y.iloc[train_indices]

best_cv_params_edited = tune_hyperparams(X_train_first_fold, y_train_first_fold, tscv_inner=tscv_inner, lags=lags, 
                                         level='store_nbr')

In [ ]:
base_pipelines = {}
for current_family in X['family'].unique():
    if current_family in linear_categories:
        base_pipelines[current_family] = LinearPipeline(cols_to_scale=['dcoilwtico'], cols_to_encode=['store_nbr'], 
                                                        drop_columns=['onpromotion'], lags=lags, split_key='family', 
                                                        target_col='sales', level='store_nbr')
    else:
        base_pipelines[current_family] = LGBMPipeline(lags=lags, split_key='family', target_col='sales', level='store_nbr',
                                                      params=best_cv_params_edited[current_family], fit_params=fit_params)


modelling_pipeline = RecursiveTSEstimatorWithZeroCategories(split_key='family', target_col='sales', 
                                                            base_pipelines=base_pipelines, 
                                                            zero_categories=['baby care', 'books'])

scores = cross_validate(modelling_pipeline, X, y, cv=splitter, scoring=CV_METRICS, return_estimator=True, error_score='raise', 
                        n_jobs=-1)
format_cv_test_scores(scores, metrics_to_plot=['root_mean_squared_log_error'])

In [ ]:
plt.figure(figsize=(30, 200))
for i, family in enumerate(X['family'].unique()):
    plt.subplot(33, 2, 2 * i + 1)
    average_sales = X[(X['family'] == family) & (X['store_nbr'] == 18)].groupby('date')['sales'].median()
    trend = average_sales.rolling(window=365, center=True, min_periods=183).mean()
    plt.ylabel('mean sales')
    ax = average_sales.plot(alpha=0.5)
    ax = trend.plot(ax=ax, linewidth=3)
    plt.title(family, color='red')
    
    plt.subplot(33, 2, 2 * i + 2)
    average_sales = X[(X['family'] == family) & (X['store_nbr'] == 25)].groupby('date')['sales'].median()
    trend = average_sales.rolling(window=365, center=True, min_periods=183).mean()
    plt.ylabel('mean sales')
    ax = average_sales.plot(alpha=0.5)
    ax = trend.plot(ax=ax, linewidth=3)
    plt.title(family, color='red')

In [ ]:
# saving the predictions of the best model

linear_categories = ['baby care', 'books', 'lawn and garden', 'home appliances']
fit_params = {'feature_name': ['store_nbr', 'onpromotion', 'dcoilwtico', 'lag_1', 'lag_2', 'lag_4', 'lag_6', 'lag_7', 'lag_14'],
              'categorical_feature': ['store_nbr']}

X_train, y_train = cut_history(X=X, date_column='date', keep_interval=pd.Timedelta(days=DAYS_IN_YEAR), y=y)

base_pipelines = {}
for current_family in X['family'].unique():
    if current_family in linear_categories:
        base_pipelines[current_family] = LinearPipeline(cols_to_scale=['dcoilwtico'], cols_to_encode=['store_nbr'], 
                                                         drop_columns=['onpromotion'], lags=lags, split_key='family', 
                                                         target_col='sales', level='store_nbr')
    else:
        base_pipelines[current_family] = LGBMPipeline(lags=lags, split_key='family', target_col='sales', level='store_nbr',
                                                       fit_params=fit_params, params=best_final_params[current_family])

final_modelling_pipeline = RecursiveTSEstimatorWithZeroCategories(split_key='family', target_col='sales', 
                                                                  base_pipelines=base_pipelines, 
                                                                  zero_categories=['baby care', 'books'])
final_modelling_pipeline.fit(X_train, y_train)
submission = pd.read_csv(DATA_ROOT / 'sample_submission.csv')
submission['sales'] = final_modelling_pipeline.predict(test_data)

In [ ]:
X_train_store_52 = X_train[(X_train['store_nbr'] == 52) & (X_train['date'] >= '2017-04-20')]
y_train_store_52 = y_train.loc[X_train_store_52.index]

In [ ]:
# hyperparameters tuning for 52-nd store_nbr

tscv_inner = TimeSeriesSplit(gap=0, n_splits=2, max_train_size=X_train_store_52['date'].nunique() - 2 * N_HORIZONS,
                             test_size=N_HORIZONS)
best_params_52 = tune_hyperparams(X_train_store_52, y_train_store_52, tscv_inner=tscv_inner, level='store_nbr', lags=lags, 
                                  drop_columns={family: ['store_nbr'] for family in X['family'].unique()})

In [ ]:
import json
        
with open(DATA_ROOT / 'best_params_52.txt') as f:
    data = f.read()
best_params_52 = json.loads(data)

with open(DATA_ROOT / 'best_cv_params_edited.txt') as f:
    data = f.read()
best_cv_params_edited = json.loads(data)

In [ ]:
# making predictions for 52-nd store_nbr

base_pipelines_52 = {}
for current_family in X['family'].unique():
    if current_family in linear_categories:
        base_pipelines_52[current_family] = LinearPipeline(cols_to_scale=['dcoilwtico'], drop_columns=['onpromotion'], 
                                                           lags=lags, split_key='family', target_col='sales', level='store_nbr')
    else:
        base_pipelines_52[current_family] = LGBMPipeline(lags=lags, split_key='family', target_col='sales', level='store_nbr',
                                                         params=best_params_52[current_family])

modelling_pipeline_52 = RecursiveTSEstimatorWithZeroCategories(split_key='family', target_col='sales', 
                                                               base_pipelines=base_pipelines_52, 
                                                               zero_categories=['baby care', 'books'])
modelling_pipeline_52.fit(X_train_store_52, y_train_store_52)
preds_store_52 = modelling_pipeline_52.predict(test_data[test_data['store_nbr'] == 52])
submission.loc[test_data[test_data['store_nbr'] == 52].index, 'sales'] = preds_store_52

In [ ]:
submission.to_csv(DATA_ROOT / 'best_model_preds.csv', index=False)

In [ ]:
scores_for_every_store_nbr = scores_for_every_feature_value(X, y, feature='store_nbr', base_pipelines=base_pipelines)
plot_scores_for_every_feature_value(scores_for_every_store_nbr, 'store_nbr')